In [15]:
#Inserire in una cartella di lavoro i file zip dei dataset
#Creazione delle directory ed estrazione dei dataset

import os
cwd = os.getcwd()
print(cwd)

#Crea directory per estrarre i dataset solo una volta all'inizio
if not os.path.exists('celeba'):
    os.makedirs('celeba')
if not os.path.exists('utkface'):
    os.makedirs('utkface')

#Estrae i due dataset solo una volta all'inizio
from zipfile import ZipFile
with ZipFile('celeba-dataset.zip', 'r') as zipObj:
    if len(os.listdir('celeba')) == 0:
        zipObj.extractall('celeba')

with ZipFile('utkface-new.zip', 'r') as zipObj:
    if len(os.listdir('utkface')) == 0:
        zipObj.extractall('utkface')

C:\Users\vince\Desktop\Progetto FVAB
